In [ ]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None # ?
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    # 나의 creatror
    def backward(self):
        f = self.creator  # 1. Get a function

        # 조상을 추적하시오.
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward

            # 재귀를 시작하시오. 
            x.backward()


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)

        # 함수 호출로 나가는 output 의 creator는 나여! 
        output.set_creator(self)  # Set parent(function)
        self.input = input
        self.output = output  # Set output
        return output

    # 위임
    def forward(self, x):
        raise NotImplementedError()

    # 위임
    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)